In [1]:
import trimesh

scene = trimesh.Scene()

In [10]:
from my_code.datasets.smpl_webuser.serialization import load_model
from my_code.datasets.smal_online.my_mesh.mesh import myMesh
import pickle as pkl

# Load the smal model 
model_path = '/home/s94zalek_hpc/shape_matching/my_code/datasets/smal_online/smal_CVPR2017.pkl'
model = load_model(model_path)

### Template

In [13]:
# Save the mean model
# model.r are the model vertexes, and model.f are the mesh faces.

template_original = trimesh.Trimesh(vertices=model.r, faces=model.f)

  

In [ ]:
import my_code.sign_canonicalization.remesh as remesh
import utils.fmap_util as fmap_util
import torch
import my_code.datasets.preprocessing as preprocessing


verts_orig = torch.tensor(model.r, dtype=torch.float32)
faces_orig = torch.tensor(model.f.astype(np.int64), dtype=torch.int64)

verts_orig = preprocessing.normalize_face_area(verts_orig, faces_orig)
      

verts_r, faces_r = remesh.remesh_simplify_iso(
    verts=verts_orig,
    faces=faces_orig,
    n_remesh_iters=10,
    remesh_targetlen=1,
    simplify_strength=1,
    )


corr_r = fmap_util.nn_query(
    verts_r,
    verts_orig, 
    )

template_remeshed = trimesh.Trimesh(
    vertices=verts_r,
    faces=faces_r
    )

scene.geometry.clear()
scene.add_geometry(trimesh.creation.axis(origin_size=0.1))

# scene.add_geometry(template_original)
scene.add_geometry(template_remeshed)

scene.set_camera()
scene.show()

In [21]:
template_remeshed.export('/home/s94zalek_hpc/shape_matching/data/SMAL_templates/remeshed/template.off')
np.savetxt('/home/s94zalek_hpc/shape_matching/data/SMAL_templates/remeshed/corr.txt', corr_r.numpy() + 1, fmt='%d')


In [ ]:
template_orig = trimesh.Trimesh(vertices=verts_orig, faces=faces_orig)
template_original.export('/home/s94zalek_hpc/shape_matching/data/SMAL_templates/original/template.off')

scene.geometry.clear()
scene.add_geometry(trimesh.creation.axis(origin_size=0.1))
scene.add_geometry(template_orig)
scene.set_camera()
scene.show()

In [2]:
import numpy as np

# original correspondences
template_orig = trimesh.load('/home/s94zalek_hpc/shape_matching/data/SMAL_templates/original/template.off', process=False)

# 1-1 correspondences
corr = range(1, len(template_orig.vertices) + 1)
np.savetxt('/home/s94zalek_hpc/shape_matching/data/SMAL_templates/original/corr.txt', corr, fmt='%d')

# corr = np.loadtxt('/home/s94zalek_hpc/shape_matching/data/SMAL_templates/remeshed/corr.txt', dtype=np.int32) - 1

### Families of animals

In [4]:
# Load the family clusters data (see paper for details)
# and save the mean per-family shape
# 0-felidae(cats); 1-canidae(dogs); 2-equidae(horses);
# 3-bovidae(cows); 4-hippopotamidae(hippos);
# The clusters are over the shape coefficients (betas);
# setting different betas changes the shape of the model

model_data_path = '/home/s94zalek_hpc/shape_matching/my_code/datasets/smal_online/smal_CVPR2017_data.pkl'
with open(model_data_path, 'rb') as model_data_path:
    data = pkl.load(model_data_path, encoding="latin1")

In [ ]:
import numpy as np

scene.geometry.clear()

for i, betas in enumerate(data['cluster_means']):
    model.betas[:] = betas
    model.pose[:] = 0.
    model.trans[:] = 0.
    # m = myMesh(v=model.r, f=model.f)
    # m.save_ply('family_'+str(i)+'.ply')
    
    mesh_i = trimesh.Trimesh(
        vertices=model.r + np.array([2*i, 0, 0]),
        faces=model.f)
    
    scene.add_geometry(mesh_i)
    
print('saved mean per-family shape')

scene.show()


In [ ]:
scene.geometry.clear()

# Show the toys in T-pose
for i, betas in enumerate(data['toys_betas']):
    model.betas[:] = betas
    model.pose[:] = 0.
    model.trans[:] = 0.
    # m = myMesh(v=model.r, f=model.f)
    # m.save_ply('toy_'+str(i)+'.ply')
    
    mesh_i = trimesh.Trimesh(
        vertices=model.r + np.array([2*i, 0, 0]),
        faces=model.f)
    
    scene.add_geometry(mesh_i)
    
print('saved toys in t-pose')

scene.show()


## Generate data

In [ ]:
betas = data['cluster_means'][4]

model.betas[:] = betas
model.pose[:] = 0.
model.trans[:] = 0.

scene.geometry.clear()

for i in range(10):
    model.pose[0:3]=0
    model.pose[3:]=0.2 * np.random.randn(96)
    
    
    # print(model.betas)
    
    # model.betas[:] = betas + 0.5 * np.random.randn(*betas.shape)
    
    print(model.r)
    
    # print(model.betas)
    
    
    
    mesh_i = trimesh.Trimesh(
        vertices=model.r + np.array([3*i, 0, 0]),
        faces=model.f)
    
    
    scene.add_geometry(mesh_i)

scene.show()
 

In [11]:
import my_code.datasets.preprocessing as preprocessing
import torch


def process_mesh(mesh):

    item = dict()

    item['verts'] = torch.from_numpy(mesh.vertices).float()
    item['faces'] = torch.from_numpy(mesh.faces).long()
            
    # item['verts'] = preprocessing.center_bbox(item['verts'])

    # normalize vertices by area
    item['verts'] = preprocessing.normalize_face_area(item['verts'], item['faces'])


    item = preprocessing.get_spectral_ops(item, num_evecs=32,
                                                    cache_dir=None)
    
    mesh_out = trimesh.Trimesh(vertices=item['verts'].numpy(), faces=item['faces'].numpy())
    
    return item, mesh_out


In [ ]:
import my_code.utils.plotting_utils as plotting_utils
import matplotlib.pyplot as plt
import torch


betas = data['cluster_means'][0]

model.betas[:] = betas
model.pose[:] = 0.
model.trans[:] = 0.


model.pose[0:3]=0
model.pose[3:]=0.2 * np.random.randn(96)
# model.betas[:] = betas + 0.5 * np.random.randn(*betas.shape)
    
mesh_1_trimesh = trimesh.Trimesh(
    vertices=model.r,
    faces=model.f)

mesh_1, mesh_1_trimesh = process_mesh(mesh_1_trimesh)




betas = data['cluster_means'][0]

model.betas[:] = betas
model.pose[:] = 0.
model.trans[:] = 0.


model.pose[0:3]=0
model.pose[3:]=0.2 * np.random.randn(96)
# model.betas[:] = betas + 0.5 * np.random.randn(*betas.shape)
    
mesh_2_trimesh = trimesh.Trimesh(
    vertices=model.r + np.array([0, 0, 0]),
    faces=model.f)

mesh_2, mesh_2_trimesh = process_mesh(mesh_2_trimesh)   




scene.geometry.clear()

plotting_utils.plot_p2p_map(
    scene,
    
    
    mesh_2['verts'], mesh_2['faces'],
    mesh_1['verts'], mesh_1['faces'],
    
    torch.arange(mesh_1['verts'].shape[0]).int(),
    axes_color_gradient=[0, 1],
    base_cmap='hsv'
)

scene.show()

In [ ]:


C_gt_xy_lstsq = torch.linalg.lstsq(
    mesh_2['evecs'],
    mesh_1['evecs']
    ).solution

l = 0
h = 32

fig, axs = plt.subplots(1, 1, figsize=(4, 4))

plotting_utils.plot_Cxy(fig, axs,  C_gt_xy_lstsq,
                        'before', l, h, show_grid=False, show_colorbar=False)
plt.show()

In [ ]:
lion_data_path = '/home/s94zalek_hpc/shape_matching/data/SMAL_fitted/lion6.pkl'

with open(lion_data_path, 'rb') as lion_data_path:
    lion_data = pkl.load(lion_data_path, encoding="latin1")
    
lion_data

### Generate other poses for each train shape

In [ ]:
train_names = '/home/s94zalek_hpc/shape_matching/data/SMAL_r/train_cat.txt'
# read the train names
with open(train_names) as f:
    train_names = f.readlines()
    train_names = [x.strip() for x in train_names]
    
    
train_names

In [ ]:
# read name map 

name_map = '/home/s94zalek_hpc/shape_matching/data/SMAL_r/name_map.txt'

with open(name_map) as f:
    name_map = f.readlines()
    name_map = [x.strip() for x in name_map]
    
    name_map = dict([x.split()[::-1] for x in name_map])
    
name_map

In [ ]:
# for train_name in train_names:
    

orig_name = name_map['cougar_01']

orig_path = f'/home/s94zalek_hpc/shape_matching/data/SMAL_fitted/{orig_name}.pkl'
with open(orig_path, 'rb') as orig_path:
    shape_data = pkl.load(orig_path, encoding="latin1")
    
    
model.betas[:] = np.zeros_like(model.betas)
model.betas[:20] = np.array(shape_data['beta']) + 0.05 * np.random.randn(*shape_data['beta'].shape)

model.pose[:] = np.array(shape_data['pose']) + 0.2 * np.random.randn(*shape_data['pose'].shape)
model.pose[0:3]=0

model.trans[:] = 0.


mesh_trimesh = trimesh.Trimesh(
    vertices=model.r,
    faces=model.f)

scene.geometry.clear()

scene.add_geometry(mesh_trimesh)


scene.set_camera()
scene.show()


    
   

In [ ]:
from tqdm import tqdm

verts_generated = []

for _ in tqdm(range(1000)):

    model.betas[:] = np.zeros_like(model.betas)
    model.betas[:20] = np.array(shape_data['beta']) + 0.05 * np.random.randn(*shape_data['beta'].shape)

    model.pose[:] = np.array(shape_data['pose']) + 0.2 * np.random.randn(*shape_data['pose'].shape)
    model.pose[0:3]=0

    model.trans[:] = 0.
    
    verts_generated.append(torch.tensor(model.r.copy()))





In [ ]:
scene.geometry.clear()


for i in range(100, 110):
    mesh_trimesh = trimesh.Trimesh(
        vertices=verts_generated[i] + torch.tensor([2*i, 0, 0]),
        faces=model.f)

    scene.add_geometry(mesh_trimesh)

scene.set_camera()
scene.show()

### Generate the train set for SignNet

In [ ]:
import os

curr_id = 0

off_folder = '/home/s94zalek_hpc/shape_matching/data_sign_training/train/SMAL_train_435/off'
os.makedirs(off_folder, exist_ok=True)

for train_name in train_names:
    
    print(train_name)

    orig_name = name_map[train_name]

    orig_path = f'/home/s94zalek_hpc/shape_matching/data/SMAL_fitted/{orig_name}.pkl'
    with open(orig_path, 'rb') as orig_path:
        shape_data = pkl.load(orig_path, encoding="latin1")
        
        
    for i in range(15):
        
        model.betas[:] = np.zeros_like(model.betas)
        model.betas[:20] = np.array(shape_data['beta']) + 0.05 * np.random.randn(*shape_data['beta'].shape)

        model.pose[:] = np.array(shape_data['pose']) + 0.2 * np.random.randn(*shape_data['pose'].shape)
        model.pose[0:3]=0

        model.trans[:] = 0.


        mesh_trimesh = trimesh.Trimesh(
            vertices=model.r,
            faces=model.f, process=False
            )

        mesh_trimesh.export(f'{off_folder}/{curr_id:04}.off')
        
        curr_id += 1
       
   

In [ ]:
# read 10 random meshes from the training set and plot them



scene.geometry.clear()

random_indices = np.random.choice(15, 10, replace=False)

for i, idx in enumerate(random_indices):
    mesh_trimesh = trimesh.load_mesh(f'{off_folder}/{idx:04}.off')
    
    mesh_trimesh.vertices += np.array([2*i, 0, 0])
    
    scene.add_geometry(mesh_trimesh)
    
    mesh_trimesh_aug = trimesh.load_mesh(f'/home/s94zalek_hpc/shape_matching/data_sign_training/train/SMAL_isoRemesh_0.2_0.8/off/{idx:04}.off')
    
    mesh_trimesh_aug.vertices += np.array([2*i, 1, 0])
    
    scene.add_geometry(mesh_trimesh_aug)
    
scene.set_camera()
scene.show()


## Generate the train set for DDPM

In [ ]:
curr_id = 0

shape_list = []

for train_name in train_names:
    
    print(train_name)

    orig_name = name_map[train_name]

    orig_path = f'/home/s94zalek_hpc/shape_matching/data/SMAL_fitted/{orig_name}.pkl'
    with open(orig_path, 'rb') as orig_path:
        shape_data = pkl.load(orig_path, encoding="latin1")
        
        
    for i in tqdm(range(7000)):
        
        model.betas[:] = np.zeros_like(model.betas)
        model.betas[:20] = np.array(shape_data['beta']) + 0.05 * np.random.randn(*shape_data['beta'].shape)

        model.pose[:] = np.array(shape_data['pose']) + 0.2 * np.random.randn(*shape_data['pose'].shape)
        model.pose[0:3]=0

        model.trans[:] = 0.

        shape_list.append(torch.tensor(model.r.copy(), dtype=torch.float32))
       
   

In [ ]:
shape_list_stacked = torch.stack(shape_list)

shape_list_stacked.shape

In [41]:
torch.save(shape_list_stacked, '/lustre/mlnvme/data/s94zalek_hpc-shape_matching/SMAL_shapes_train.pt')

In [ ]:
import torch
import numpy as np

# read 10 random meshes from the training set and plot them
scene.geometry.clear()

random_indices = np.random.choice(smal_data.shape[0], 10, replace=False)

for i, idx in enumerate(random_indices):
    mesh_trimesh = trimesh.Trimesh(
        vertices=smal_data[idx].numpy(),
        faces=model.f)
    
    mesh_trimesh.vertices += np.array([2*i, 0, 0])
    
    scene.add_geometry(mesh_trimesh)
    
scene.set_camera()
scene.show()


### Test template dataset

In [2]:
from my_code.datasets.surreal_dataset_3dc import TemplateSurrealDataset3DC
import numpy as np

augmentations = {
            "remesh": {
                "isotropic": {
                    "n_remesh_iters": 10,
                    "remesh_targetlen": 1,
                    "simplify_strength_min": 0.2,
                    "simplify_strength_max": 0.8,
                },
                "anisotropic": {
                    "probability": 0.35,
                        
                    "n_remesh_iters": 10,
                    "fraction_to_simplify_min": 0.2,
                    "fraction_to_simplify_max": 0.6,
                    "simplify_strength_min": 0.2,
                    "simplify_strength_max": 0.5,
                    "weighted_by": "face_count",
                },
            },
        }

dataset = TemplateSurrealDataset3DC(
            shape_path='/lustre/mlnvme/data/s94zalek_hpc-shape_matching/SMAL_shapes_train_32.pt',
            num_evecs=128,
            cache_lb_dir=None,
            return_evecs=True,
            return_fmap=False,
            mmap=True,
            augmentations=augmentations,
            template_path=f'/home/s94zalek_hpc/shape_matching/data/SMAL_templates/remeshed/template.off',
            template_corr=np.loadtxt(
                f'/home/s94zalek_hpc/shape_matching/data/SMAL_templates/remeshed/corr.txt',
                dtype=np.int32) - 1,
            mesh_orig_faces_path='/home/s94zalek_hpc/shape_matching/data/SMAL_templates/original/template.off',
            centering='bbox',
            return_shot=False,
        )  

In [ ]:
scene.geometry.clear()

data_i = dataset[30000]

mesh_1 = trimesh.Trimesh(
    data_i['first']['verts'].numpy(),
    data_i['first']['faces'].numpy()
)
mesh_2 = trimesh.Trimesh(
    data_i['second']['verts'].numpy() + np.array([1, 0, 0]),
    data_i['second']['faces'].numpy()
)

scene.add_geometry(mesh_1)
scene.add_geometry(mesh_2)

scene.set_camera()
scene.show()

In [ ]:
data_i['second']['corr'].dtype

In [ ]:
import os
import torch

file = torch.load('/lustre/mlnvme/data/s94zalek_hpc-shape_matching/SURREAL/train/SMAL_32_SMAL_isoRemesh_0.2_0.8_50000_aug/train/C_gt_xy_2030_4060.pt')